In [ ]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
# !pip install cliffs_delta
from cliffs_delta import cliffs_delta
from scipy.stats import wilcoxon
from numpy import median,mean
from tqdm import tqdm
import sqlite3,os,re
from IPython.display import display
from datetime import datetime

In [ ]:
cate = {'Ac':'Accessibility',
'Bl':'Blogging',
'De':'Developer Tools',
'Fu':'Fun',
'Ne':'News & Weather',
'Ph':'Photos',
'Pr':'Productivity',
'Se':'Search Tools',
'Sh':'Shopping',
'So':'Social & Communication',
'Sp':'Sports'}
extensions = {
    #12
'fmidkjgknpkbmninbmklhcgaalfalbdh':{'category':'Ac','label':0,'pp':1,'target':'Generic'},
'ffnhmkgpdmkajhomnckhabkfeakhcamm':{'category':'Bl','label':0,'pp':1,'target':'Generic'},
'gppongmhjkpfnbhagpmjfkannfbllamg':{'category':'De','label':0,'pp':1,'target':'Generic'},
'iginnfkhmmfhlkagcmpgofnjhanpmklb':{'category':'Fu','label':0,'pp':1,'target':'Generic'},
'kpghljlpdknmomchobaoecdlkcpocaga':{'category':'Ne','label':0,'pp':1,'target':'News'},
'lamfengpphafgjdgacmmnpakdphmjlji':{'category':'Ph','label':0,'pp':1,'target':'Generic'},
'gpdjojdkbbmdfjfahjcgigfpmkopogic':{'category':'Pr','label':0,'pp':1,'target':'Generic'},
'bmhcbmnbenmcecpmpepghooflbehcack':{'category':'Se','label':0,'pp':1,'target':'Generic'},
'pgmbeccjfkdbpdjfoldaahpfamjjafma':{'category':'Sh','label':0,'pp':1,'target':'Shoppingjp'},
'cimpffimgeipdhnhjohpbehjkcdpjolg':{'category':'So','label':0,'pp':1,'target':'Video_youtube'},
'ahcblhpcealjpkmndgmkdnebbjakicno':{'category':'Sp','label':0,'pp':1,'target':'Espn'},
    'pgniedifoejifjkndekolimjeclnokkb':{'category':'Fu','label':0,'pp':1,'target':'Video_twitch'},
    
    #11
'gmopgnhbhiniibbiilmbjilcmgaocokj':{'category':'Ac','label':1,'pp':2,'target':'Video_youtube'},
'ffjnfifmelbmglnajefiipdeejghkkjg':{'category':'Bl','label':1,'pp':2,'target':'Generic'},
'bkhaagjahfmjljalopjnoealnfndnagc':{'category':'De','label':1,'pp':2,'target':'GitHub'},
'ikdgincnppajmpmnhfheflannaiapmlm':{'category':'Fu','label':1,'pp':2,'target':'Generic'},
'kfimphpokifbjgmjflanmfeppcjimgah':{'category':'Ne','label':1,'pp':2,'target':'Generic'},
'lcpkicdemehhmkjolekhlglljnkggfcf':{'category':'Ph','label':1,'pp':1,'target':'Generic'},
'nngceckbapebfimnlniiiahkandclblb':{'category':'Pr','label':1,'pp':2,'target':'Generic'},
'naankklphfojljboaokgfbheobbgenka':{'category':'Se','label':1,'pp':2,'target':'Generic'},
'fhjanlpjlfhhbhbnjohflphmfccbhmoi':{'category':'Sh','label':1,'pp':2,'target':'Shoppingjp'},
'bkkjeefjfjcfdfifddmkdmcpmaakmelp':{'category':'So','label':1,'pp':2,'target':'Video_youtube'},
'lokmacldfjfgajcebibmmfohacnikhhd':{'category':'Sp','label':1,'pp':1,'target':'Draftkings'},
    #13
'alncdjedloppbablonallfbkeiknmkdi':{'category':'Ac','label':2,'pp':1,'target':'Generic'},
'pkihbahhbihfoebgdfkibnblbhjfgefc':{'category':'Bl','label':2,'pp':1,'target':'Apk'},
'dapjbgnjinbpoindlpdmhochffioedbn':{'category':'De','label':2,'pp':1,'target':'Generic'},
'fadndhdgpmmaapbmfcknlfgcflmmmieb':{'category':'Fu','label':2,'pp':1,'target':'Video_twitch'},
'iolcbmjhmpdheggkocibajddahbeiglb':{'category':'Ne','label':2,'pp':1,'target':'Generic'},
'mpejojclnbakefnlfmnkaaianojbicdk':{'category':'Ph','label':2,'pp':1,'target':'Generic'},
'mmeijimgabbpbgpdklnllpncmdofkcpn':{'category':'Pr','label':2,'pp':1,'target':'Generic'},
'aookogakccicaoigoofnnmeclkignpdk':{'category':'Se','label':2,'pp':1,'target':'Generic'},
'neebplgakaahbhdphmkckjjcegoiijjo':{'category':'Sh','label':2,'pp':1,'target':'Shoppingjp'},
'jicldjademmddamblmdllfneeaeeclik':{'category':'So','label':2,'pp':1,'target':'Oktool'},
'mlfkmhibffpoleieiomjkekmjipdekhg':{'category':'Sp','label':2,'pp':1,'target':'Video_youtube'},
    'jiiidpmjdakhbgkbdchmhmnfbdebfnhp':{'category':'De','label':2,'pp':1,'target':'Generic'},
    'behkgahlidmeemjefcbgieigiejiglpc':{'category':'Ne','label':2,'pp':1,'target':'Generic'},
    #11
'kammdlphdfejlopponbapgpbgakimokm':{'category':'Ac','label':3,'pp':2,'target':'Generic'},
'fnhmjceoafkkibpijbfpfajbhkknadmb':{'category':'Bl','label':3,'pp':4,'target':'Tricky'},
'fklgmciohehgadlafhljjhgdojfjihhk':{'category':'De','label':3,'pp':2,'target':'Generic'},
'jpefkkpmalfnilnbghfnjodceifpemdb':{'category':'Fu','label':3,'pp':2,'target':'Ereality'},
'pgfokhpgehbmeifbpdhegfnpaahabfja':{'category':'Ne','label':3,'pp':2,'target':'News'},
'aiiimepjikpdipbpmknolbnjbeohbmaa':{'category':'Ph','label':3,'pp':2,'target':'Generic'},
'jbebkmmlkhioeagiekpopmeecaepaihd':{'category':'Pr','label':3,'pp':2,'target':'Generic'},
'didkfdopbffjkpolefhpcjkohcpalicd':{'category':'Se','label':3,'pp':2,'target':'Generic'},
'pbjikboenpfhbbejgkoklgkhjpfogcam':{'category':'Sh','label':3,'pp':3,'target':'Shoppingjp'},
'mhkhmbddkmdggbhaaaodilponhnccicb':{'category':'So','label':3,'pp':2,'target':'Video_youtube'},
'lipplpkgbnhdfdchoibgafjdblpjdkpi':{'category':'Sp','label':3,'pp':3,'target':'Lichess'},
    #14
'bkpenclhmiealbebdopglffmfdiilejc':{'category':'Ac','label':4,'pp':1,'target':'Generic'},
'lnphplhkejidgcncalbkbngbiafmjnml':{'category':'Bl','label':4,'pp':1,'target':'Generic'},
'dgjhfomjieaadpoljlnidmbgkdffpack':{'category':'De','label':4,'pp':1,'target':'GitHub'},
'hcbddpppkcnfjifbcfnhmelpemdoepkk':{'category':'Fu','label':4,'pp':1,'target':'Generic'},
'kgaebnfbgpcnglnhjhglinfiecgccfij':{'category':'Ne','label':4,'pp':1,'target':'Generic'},
'iedjpcecgmldlnkbojiocmdaedhepbpn':{'category':'Ph','label':4,'pp':1,'target':'Generic'},
'nkbihfbeogaeaoehlefnkodbefgpgknn':{'category':'Pr','label':4,'pp':1,'target':'Generic'},
'eedlgdlajadkbbjoobobefphmfkcchfk':{'category':'Se','label':4,'pp':1,'target':'Generic'},
'gfkpklgmocbcbdabfellcnikamdaeajd':{'category':'Sh','label':4,'pp':2,'target':'Shoppingjp'},
'bboamecjefgpaemgfpcjeediamdnkklc':{'category':'So','label':4,'pp':1,'target':'Video_youtube'},
'kimjfkgkpmafgngclkdpjdlkdlghoikh':{'category':'Sp','label':4,'pp':1,'target':'Espn'},
    'bhggankplfegmjjngfmhfajedmiikolo':{'category':'Ac','label':4,'pp':1,'target':'Generic'},
    'ldjnabbinoccbodkejkdiolmadimbjkj':{'category':'Bl','label':4,'pp':2,'target':'Jobsalert'},
    'obhadkdgdffnnbdfpigjklinjhbkinfh':{'category':'Fu','label':4,'pp':2,'target':'Shadowpay'}
             }

# Normalization

## Extension-free normalization

In [ ]:
f = pd.read_csv('/xxxxxxxx/free.txt', sep=" ", header=None)[0]
free_cycle = pd.DataFrame()

for i in range(int(len(f)/7)):
    type = f[i*7]
    timestamp = f[i*7+1]
    time = f[i*7+2]
    core = f[i*7+3]
    dram = f[i*7+4]
    core5 = f[i*7+5]
    dram5 = f[i*7+6]
    loading = float(core)+float(dram)
    static = float(core5)+float(dram5)
    tmp = {'type':type,'time':time,'loading':loading,'static':static}
    free_cycle = pd.concat([free_cycle, pd.DataFrame([tmp])], ignore_index=True)

free = {'normal':{}}

for type in list(dict.fromkeys(free_cycle['type'])):
    time = free_cycle[free_cycle['type']==type]['time'].astype('float64')
    loading = free_cycle[free_cycle['type']==type]['loading'].astype('float64')
    static = free_cycle[free_cycle['type']==type]['static'].astype('float64')
    
    batch = 10
    batch_times = 10
    for i in range(batch):
        
        # # x/median Normalization
        # #  Normalize website by core/dram/...
        time.iloc[i*batch:i*batch+batch_times] = time[i*batch:i*batch+batch_times]/time[i*batch:i*batch+batch_times].median()
        loading.iloc[i*batch:i*batch+batch_times] = loading[i*batch:i*batch+batch_times]/loading[i*batch:i*batch+batch_times].median()
        static.iloc[i*batch:i*batch+batch_times] = static[i*batch:i*batch+batch_times]/static[i*batch:i*batch+batch_times].median()
        
       
    free['normal'][type] = {'time':time,'loading':loading,'static':static}
    
    Ntime = []
    Ncore = []
    Ndram = []
    Ncore5 = []
    Ndram5 = []
    Nloading = []
    Nstatic = []
    Ntotal = []
    for i in range(batch):
        Ntime.append(free['normal'][type]['time'][i*batch:i*batch+batch_times].mean())
        Nloading.append(free['normal'][type]['loading'][i*batch:i*batch+batch_times].mean())
        Nstatic.append(free['normal'][type]['static'][i*batch:i*batch+batch_times].mean())

free['normal']

## Normalization - fully loaded mode

In [ ]:
f = pd.read_csv('/xxxxxxxx/full.txt', sep=" ", header=None)[0]
# id = []
full = pd.DataFrame()
for i in range(int(len(f)/7)):
    id = f[i*7]
    timestamp = f[i*7+1]
    time = f[i*7+2]
    core = f[i*7+3]
    dram = f[i*7+4]
    core5 = f[i*7+5]
    dram5 = f[i*7+6]
    loading = float(core)+float(dram)
    static = float(core5)+float(dram5)
    total = float(loading) + float(static)
    target=extensions[id]['target']
    tmp = {'id':id,
           'category':cate[extensions[id]['category']],
           'label':extensions[id]['label'],
           '#pp':extensions[id]['pp'],
           'target':target.lower(),
           # 'timestamp':timestamp,
           'otime':time,
           'ocore':core,'odram':dram,'ocore5':core5,'odram5':dram5,
           'oloading':loading,'ostatic':static,'ototal': total
          }
    full = pd.concat([full, pd.DataFrame([tmp])], ignore_index=True)  

for type in list(dict.fromkeys(full['target'])):
    id_list = list(dict.fromkeys(full[full['target']==type]['id']))
    
    time_all = full[full['target']==type]
    core_all = full[full['target']==type]
    dram_all = full[full['target']==type]
    core5_all = full[full['target']==type]
    dram5_all = full[full['target']==type]
    
    
    time_base = free_cycle[free_cycle['type']==type]['time'].astype('float64')
    core_base = free_cycle[free_cycle['type']==type]['core'].astype('float64')
    dram_base = free_cycle[free_cycle['type']==type]['dram'].astype('float64')
    core5_base = free_cycle[free_cycle['type']==type]['core5'].astype('float64')
    dram5_base = free_cycle[free_cycle['type']==type]['dram5'].astype('float64')
    
    loading_base = free_cycle[free_cycle['type']==type]['loading'].astype('float64')
    static_base = free_cycle[free_cycle['type']==type]['static'].astype('float64')
    total_base = free_cycle[free_cycle['type']==type]['total'].astype('float64')
    # # extensions in different web types
    for id in id_list:
        idx = full.index[full['id']==id]
        
        batch = idx[0]
        batch_times = 10
        for i in range(batch_times):
            
            # # Normalize x/median by website based on free version
            full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'time'] = full[full['id']==id]['otime'][i*batch_times:i*batch_times+batch_times].astype('float64')/time_base[i*batch_times:i*batch_times+batch_times].median()
            full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'core'] = full[full['id']==id]['ocore'][i*batch_times:i*batch_times+batch_times].astype('float64')/core_base[i*batch_times:i*batch_times+batch_times].median()
            full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'dram'] = full[full['id']==id]['odram'][i*batch_times:i*batch_times+batch_times].astype('float64')/dram_base[i*batch_times:i*batch_times+batch_times].median()
            full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'core5'] = full[full['id']==id]['ocore5'][i*batch_times:i*batch_times+batch_times].astype('float64')/core5_base[i*batch_times:i*batch_times+batch_times].median()
            full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'dram5'] = full[full['id']==id]['odram5'][i*batch_times:i*batch_times+batch_times].astype('float64')/dram5_base[i*batch_times:i*batch_times+batch_times].median()
            
            full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'loading'] = full[full['id']==id]['oloading'][i*batch_times:i*batch_times+batch_times].astype('float64')/loading_base[i*batch_times:i*batch_times+batch_times].median()
            full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'static'] = full[full['id']==id]['ostatic'][i*batch_times:i*batch_times+batch_times].astype('float64')/static_base[i*batch_times:i*batch_times+batch_times].median()
            full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'total'] = full[full['id']==id]['ototal'][i*batch_times:i*batch_times+batch_times].astype('float64')/total_base[i*batch_times:i*batch_times+batch_times].median()
    
print(len(full[full.time.isna() | full.core.isna() | full.dram.isna() | full.core5.isna() | full.dram5.isna() | full.loading.isna() | full.static.isna() | full.total.isna()]))
full.sort_values(by=['label','category'])

## Change ratio - fully loaded mode

In [ ]:
full_cycle = pd.DataFrame()
for id in list(dict.fromkeys(full['id'])):
    target =full[full['id']==id]['target'].tolist()[0].lower()
    
    otime = full[full['id']==id]['otime'].astype('float64')
    oloading = full[full['id']==id]['oloading'].astype('float64')
    ostatic = full[full['id']==id]['ostatic'].astype('float64')
    
    time = full[full['id']==id]['time'].astype('float64')
    loading = full[full['id']==id]['loading'].astype('float64')
    static = full[full['id']==id]['static'].astype('float64')
    
    batch = 10
    batch_times = 10
    timeRatio = []
    loadingRatio = []
    staticRatio = []
    
    for i in range(batch):
        timeRatio.append((time.iloc[i*batch:i*batch+batch_times].median()-free['normal'][type]['time'][i*batch:i*batch+batch_times].median())/(free['normal'][type]['time'][i*batch:i*batch+batch_times]).median())
        loadingRatio.append((loading.iloc[i*batch:i*batch+batch_times].median()-free['normal'][type]['loading'][i*batch:i*batch+batch_times].median())/(free['normal'][type]['loading'][i*batch:i*batch+batch_times]).median())
        staticRatio.append((static.iloc[i*batch:i*batch+batch_times].median()-free['normal'][type]['static'][i*batch:i*batch+batch_times].median())/(free['normal'][type]['static'][i*batch:i*batch+batch_times]).median())

    tmp = {'id':id,
           'category':cate[extensions[id]['category']],
           'label':extensions[id]['label'],
           '#pp':extensions[id]['pp'],
           'target':target,
           'Median-timeRatio':mean(timeRatio)*100,
           'Median-loadingRatio':mean(loadingRatio)*100,
           'Median-staticRatio':mean(staticRatio)*100,
          }
    full_cycle = pd.concat([full_cycle, pd.DataFrame([tmp])], ignore_index=True) 

full_cycle.sort_values(by=['label','category'])

## Statistically tests for the fully loaded mode

In [ ]:
def percChange(full_cycle2, interp):
    if interp == "all":
        # print("Small Cliff's d")
        printPercChange(full_cycle2, 0.147,1.1)
    if interp == "small":
        # print("Small Cliff's d")
        printPercChange(full_cycle2, 0.147,0.330)
    if interp == "median":
        # print("Median Cliff's d")
        printPercChange(full_cycle2, 0.330,0.474)
    if interp == "large":
        # print("Large Cliff's d")
        printPercChange(full_cycle2, 0.474,1.1)
            
def printPercChange(full_cycle2, lf, rt):
    for typ in ['time','loading','static']:
        numI = len(full_cycle2[(full_cycle2[f'Median-{typ}Ratio']>0) & (full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))])
        num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))])
        perc = round(numI/num*100,2) if num!=0 else 0
        
        print(len(full_cycle2[(full_cycle2[f'Median-{typ}Ratio']>0) & (full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))]), f' ({perc}%) {typ} in median increase by',
              round((full_cycle2[(full_cycle2[f'Median-{typ}Ratio']>0) & (full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))][f'Median-{typ}Ratio'].median()),2), '%')
        numI = len(full_cycle2[(full_cycle2[f'Median-{typ}Ratio']<0) & (full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))])
        num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))])
        perc = round(numI/num*100,2) if num!=0 else 0
        
        print(len(full_cycle2[(full_cycle2[f'Median-{typ}Ratio']<0) & (full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))]), f' ({perc}%) {typ} in median drop by',
              round((full_cycle2[(full_cycle2[f'Median-{typ}Ratio']<0) & (full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))][f'Median-{typ}Ratio'].median()),2), '%')
        print(len(full_cycle2[(full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))]), f' {typ} in median changes by',
              round((full_cycle2[(full_cycle2[f'wilcoxon-{typ}']<0.05) & (full_cycle2[f'cliffs_d-{typ}'].between(left=lf, right=rt, inclusive="left"))][f'Median-{typ}Ratio'].median()),2), '%')
        print('\n')
        
print('Wilcoxon + cliff d')
for cd in range(4):
    print("Small Cliff's d \n") if cd ==1 else ''
    print("Median Cliff's d \n") if cd ==2 else ""
    print("Large Cliff's d \n") if cd ==3 else ''
    
    full_cycle2 = full_cycle.copy()
    for typ in ['time','loading','static']:
        for ext in list(full_cycle2['id']):
            target =full[full['id']==ext]['target'].tolist()[0].lower()
            s,y = mannwhitneyu(full[full['id']==ext][typ].astype('float64'),free['normal'][target][typ].astype('float64'))#,method='exact')
            ws,wy = wilcoxon(full[full['id']==ext][typ].astype('float64'),free['normal'][target][typ].astype('float64'))
            d,res = cliffs_delta(full[full['id']==ext][typ].astype('float64'),free['normal'][target][typ].astype('float64'))

            idx = full_cycle2.index[full_cycle2['id']==ext][0]
            full_cycle2.at[idx,f'wilcoxon-{typ}'] = round(wy,2)
            full_cycle2.at[idx,f'cliffs_d-{typ}'] = round(abs(d),2)

    percChange(full_cycle2, 'all') if cd ==0 else ''
    percChange(full_cycle2, 'small') if cd ==1 else ''
    percChange(full_cycle2, 'median') if cd ==2 else ""
    percChange(full_cycle2, 'large') if cd ==3 else ''    

## Generalize the preprocessing procedures for other modes

In [ ]:
def read_file(filename):
    print(filename)
    f = pd.read_csv(f'/xxxxxxxx/{filename}.txt', sep=" ", header=None)[0]
    notGrant = pd.DataFrame()
    for i in range(int(len(f)/7)):
        id = f[i*7]
        timestamp = f[i*7+1]
        time = f[i*7+2]
        core = f[i*7+3]
        dram = f[i*7+4]
        core5 = f[i*7+5]
        dram5 = f[i*7+6]
        loading = float(core)+float(dram)
        static = float(core5)+float(dram5)
        target=extensions[id]['target']
        tmp = {'id':id,
               'category':cate[extensions[id]['category']],
               'label':extensions[id]['label'],
               'pp':extensions[id]['pp'],
               'target':target.lower() if filename != 'inactive' else 'generic',
               'otime':time,
               'oloading':loading,'ostatic':static
              }
        notGrant = pd.concat([notGrant, pd.DataFrame([tmp])], ignore_index=True) 
    print( len(dict.fromkeys(notGrant['id'])),'in total')
    return notGrant

def normalization(full):
    for type in list(dict.fromkeys(full['target'])):
        id_list = list(dict.fromkeys(full[full['target']==type]['id']))


        time_base = free_cycle[free_cycle['type']==type]['time'].astype('float64')
        loading_base = free_cycle[free_cycle['type']==type]['loading'].astype('float64')
        static_base = free_cycle[free_cycle['type']==type]['static'].astype('float64')
        # # extensions in different web types
        for id in id_list:
            idx = full.index[full['id']==id]

            batch = idx[0]
            batch_times = 10
            for i in range(batch_times):

                # # Normalize x/median by website based on free version
                full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'time'] = full[full['id']==id]['otime'][i*batch_times:i*batch_times+batch_times].astype('float64')/time_base[i*batch_times:i*batch_times+batch_times].median()
                full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'loading'] = full[full['id']==id]['oloading'][i*batch_times:i*batch_times+batch_times].astype('float64')/loading_base[i*batch_times:i*batch_times+batch_times].median()
                full.loc[batch+i*batch_times:batch+i*batch_times+batch_times-1,'static'] = full[full['id']==id]['ostatic'][i*batch_times:i*batch_times+batch_times].astype('float64')/static_base[i*batch_times:i*batch_times+batch_times].median()
    return full
def stat_test(full_cycle2, full,fullload,free):
        
    percChange2base(full_cycle2, full,fullload,free)
    return full_cycle2

def percChange2base(full_cycle2, full,fullload,free):
    
    print('Based on free version')
    print('Wilcoxon + cliff d')
    for typ in ['time','loading','static']:
        for ext in list(full_cycle2['id']):
            target =full[full['id']==ext]['target'].tolist()[0].lower()
            # # Ext 2 Base
            ws,wy = wilcoxon(full[full['id']==ext][typ].astype('float64'),free['normal'][target][typ].astype('float64'))
            d,res = cliffs_delta(full[full['id']==ext][typ].astype('float64'),free['normal'][target][typ].astype('float64'))

            idx = full_cycle2.index[full_cycle2['id']==ext][0]
            full_cycle2.at[idx,f'wilcoxon-{typ}2base'] = round(wy,2)
            full_cycle2.at[idx,f'cliffs_d-{typ}2base'] = round(abs(d),2)

    for typ in ['time','loading','static']:
        typ2 = typ
        lf=0.147
        rt=1.1
        
        numI =len(full_cycle2[(full_cycle2[f'{typ}2base']>0) & (full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))])
        num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))])
        perc = round(numI/num*100,2) if num!=0 else 0

        print(numI, f'({perc} %)', f' {typ} in median increase by',
              round((full_cycle2[(full_cycle2[f'{typ}2base']>0) & (full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2base'].median()),2), '%')

        numD =len(full_cycle2[(full_cycle2[f'{typ}2base']<0) & (full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))])
        num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))])
        perc = round(numD/num*100,2) if num!=0 else 0

        print(numD, f'({perc} %)', f' {typ} in median drop by',
              round((full_cycle2[(full_cycle2[f'{typ}2base']<0) & (full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2base'].median()),2), '%')
    
        print(len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))]), f' {typ} in median changes by',
              round((full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2base'].median()),2), '%')
        print('\n')
            
    for cd in range(3):
        if cd ==0:
            lf=0.147
            rt=0.330
            print("Small Cliff's d \n")
        if cd ==1:
            lf=0.330 
            rt=0.474
            print("Median Cliff's d \n")
        if cd ==2:
            lf=0.474 
            rt=1.1
            print("Large Cliff's d \n")
    
        for typ in ['time','loading','static']:
            typ2 = typ
            numI =len(full_cycle2[(full_cycle2[f'{typ}2base']>0) & (full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))])
            num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))])
            perc = round(numI/num*100,2) if num!=0 else 0

            print(numI, f'({perc} %)', f' {typ} in median increase by',
                  round((full_cycle2[(full_cycle2[f'{typ}2base']>0) & (full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2base'].median()),2), '%')
            
            # typ2 = typ.replace('Static','5') if 'Static' in typ else typ
            numD =len(full_cycle2[(full_cycle2[f'{typ}2base']<0) & (full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))])
            num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))])
            perc = round(numD/num*100,2) if num!=0 else 0

            print(numD, f'({perc} %)', f' {typ} in median drop by',
                  round((full_cycle2[(full_cycle2[f'{typ}2base']<0) & (full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2base'].median()),2), '%')
            print(len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))]), f' {typ} in median changes by',
                  round((full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2base']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2base'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2base'].median()),2), '%')
            print('\n')
        print('\n')
        
def percChange2full(full_cycle2,full,fullload,free):
    print('Based on full-loaded extension consumption')
    print('Wilcoxon + cliff d')
    for typ in ['time','loading','static']:
        for ext in list(full_cycle2['id']):
            target =full[full['id']==ext]['target'].tolist()[0].lower()
            # # Ext 2 Base
            ws,wy = wilcoxon(full[full['id']==ext][typ].astype('float64'),fullload[fullload['id']==ext][typ].astype('float64'))
            d,res = cliffs_delta(full[full['id']==ext][typ].astype('float64'),fullload[fullload['id']==ext][typ].astype('float64'))

            idx = full_cycle2.index[full_cycle2['id']==ext][0]
            full_cycle2.at[idx,f'wilcoxon-{typ}2full'] = round(wy,2)
            full_cycle2.at[idx,f'cliffs_d-{typ}2full'] = round(abs(d),2)

    for typ in ['time','loading','static']:
        lf=0.147
        rt=1.1

        typ2 = typ
        numI = len(full_cycle2[(full_cycle2[f'{typ}2full']>0) & (full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))])
        num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))])
        perc = round(numI/num*100,2) if num!=0 else 0
        print(numI, f'({perc} %)', f' {typ} in median increase by',
              round((full_cycle2[(full_cycle2[f'{typ}2full']>0) & (full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2full'].median()),2), '%')

        numD = len(full_cycle2[(full_cycle2[f'{typ}2full']<0) & (full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))])
        num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))])
        perc = round(numD/num*100,2) if num!=0 else 0
        print(numD, f'({perc} %)', f' {typ} in median drop by',
              round((full_cycle2[(full_cycle2[f'{typ}2full']<0) & (full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2full'].median()),2), '%')

        print(len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))]),f' {typ} in median changes by',
              round((full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2full'].median()),2), '%')
        print('\n')
            
    for cd in range(3):
        if cd ==0:
            lf=0.147
            rt=0.330
            print("Small Cliff's d \n")
        if cd ==1:
            lf=0.330 
            rt=0.474
            print("Median Cliff's d \n")
        if cd ==2:
            lf=0.474 
            rt=1.1
            print("Large Cliff's d \n")
        for typ in ['time','loading','static']:

            typ2 = typ
            numI = len(full_cycle2[(full_cycle2[f'{typ}2full']>0) & (full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))])
            num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))])
            perc = round(numI/num*100,2) if num!=0 else 0
            print(numI, f'({perc} %)', f' {typ} in median increase by',
                  round((full_cycle2[(full_cycle2[f'{typ}2full']>0) & (full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2full'].median()),2), '%')

            numD = len(full_cycle2[(full_cycle2[f'{typ}2full']<0) & (full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))])
            num = len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))])
            perc = round(numD/num*100,2) if num!=0 else 0
            print(numD, f'({perc} %)', f' {typ} in median drop by',
                  round((full_cycle2[(full_cycle2[f'{typ}2full']<0) & (full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2full'].median()),2), '%')
            print(len(full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))]),f' {typ} in median changes by',
                  round((full_cycle2[(full_cycle2[f'wilcoxon-{typ2}2full']<0.05) & (full_cycle2[f'cliffs_d-{typ2}2full'].between(left=lf, right=rt, inclusive="left"))][f'{typ}2full'].median()),2), '%')
            print('\n')
        print('\n')

def getRatio(notGrant,full,free):
    notGrant_cycle = pd.DataFrame()
    for id in list(dict.fromkeys(notGrant['id'])):
        target =notGrant[notGrant['id']==id]['target'].tolist()[0].lower()

        time = notGrant[notGrant['id']==id]['time'].astype('float64')
        loading = notGrant[notGrant['id']==id]['loading'].astype('float64')
        static = notGrant[notGrant['id']==id]['static'].astype('float64')
        
        batch = 10
        batch_times = 10

        time2base = []
        loading2base = []
        static2base = []

        for i in range(batch):
            time2base.append((time.iloc[i*batch:i*batch+batch_times].median()-free['normal'][type]['time'][i*batch:i*batch+batch_times].median())/(free['normal'][type]['time'][i*batch:i*batch+batch_times]).median())
            loading2base.append((loading.iloc[i*batch:i*batch+batch_times].median()-free['normal'][type]['loading'][i*batch:i*batch+batch_times].median())/(free['normal'][type]['loading'][i*batch:i*batch+batch_times]).median())
            static2base.append((static.iloc[i*batch:i*batch+batch_times].median()-free['normal'][type]['static'][i*batch:i*batch+batch_times].median())/(free['normal'][type]['static'][i*batch:i*batch+batch_times]).median())
        
        tmp = {'id':id,
               'category':cate[extensions[id]['category']],
               'label':extensions[id]['label'],
               '#pp':extensions[id]['pp'],
               'target':target,
               'time2base':mean(time2base)*100,
               'loading2base':mean(loading2base)*100,
               'static2base':mean(static2base)*100,
              }
        notGrant_cycle = pd.concat([notGrant_cycle, pd.DataFrame([tmp])], ignore_index=True)
    return notGrant_cycle

## Normalization - no Grant

In [ ]:
notGrant=read_file('notGrant')
notGrant = normalization(notGrant)
notGrant_cycle = getRatio(notGrant,full,free)
notGrant_cycle = stat_test(notGrant_cycle, notGrant,full,free)

## Normalization - no Login

In [ ]:
notLogin=read_file('notLogin')
notLogin = normalization(notLogin)
notLogin_cycle = getRatio(notLogin,full,free)
notLogin_cycle = stat_test(notLogin_cycle, notLogin,full,free)

## Normalization - Inactive

In [ ]:
inactive=read_file('inactive')
inactive = normalization(inactive)
inactive_cycle = getRatio(inactive,full,free)
inactive_cycle = stat_test(inactive_cycle, inactive,full,free)

## Normalization - Fully inactive

In [ ]:
fullinactive=read_file('fullyInactive')
fullinactive = normalization(fullinactive)
fullinactive_cycle = getRatio(fullinactive,full,free)
fullinactive_cycle = stat_test(fullinactive_cycle, fullinactive,full,free)

# Export the extension measurement to an Excel (!will be used to build the linear mixed effects model!)

In [ ]:
def user_convert(u):
    if '+' in u:
        return int(float(u.split('+')[0].strip().replace(',','')))
    else: return int(float(u.split(' ')[0].strip().replace(',','')))

def size_convert(s):
    scaler = {'KiB': 1,'MiB': 1024}
    s = re.split('([-+]?\d+\.\d+)|([-+]?\d+)',s.strip())
    s = [r.strip() for r in s if r is not None and r.strip() != '']
    return float(s[0])*scaler[s[1]]

active_NG = ['kpghljlpdknmomchobaoecdlkcpocaga', 'bkkjeefjfjcfdfifddmkdmcpmaakmelp', 'gmopgnhbhiniibbiilmbjilcmgaocokj',
'neebplgakaahbhdphmkckjjcegoiijjo', 'fadndhdgpmmaapbmfcknlfgcflmmmieb', 'pkihbahhbihfoebgdfkibnblbhjfgefc',
'jicldjademmddamblmdllfneeaeeclik', 'fnhmjceoafkkibpijbfpfajbhkknadmb', 'mhkhmbddkmdggbhaaaodilponhnccicb',
'lipplpkgbnhdfdchoibgafjdblpjdkpi', 'kimjfkgkpmafgngclkdpjdlkdlghoikh', 'obhadkdgdffnnbdfpigjklinjhbkinfh']

# grant needed
isGrant =['ffnhmkgpdmkajhomnckhabkfeakhcamm', 'fmidkjgknpkbmninbmklhcgaalfalbdh','kfimphpokifbjgmjflanmfeppcjimgah', 
'lcpkicdemehhmkjolekhlglljnkggfcf', 'fhjanlpjlfhhbhbnjohflphmfccbhmoi','naankklphfojljboaokgfbheobbgenka', 
'mlfkmhibffpoleieiomjkekmjipdekhg','bboamecjefgpaemgfpcjeediamdnkklc', 'eedlgdlajadkbbjoobobefphmfkcchfk', 
'iedjpcecgmldlnkbojiocmdaedhepbpn']

# login needed
isLogin =['ffnhmkgpdmkajhomnckhabkfeakhcamm','cimpffimgeipdhnhjohpbehjkcdpjolg', 'nngceckbapebfimnlniiiahkandclblb',
 'kfimphpokifbjgmjflanmfeppcjimgah','lokmacldfjfgajcebibmmfohacnikhhd', 'fhjanlpjlfhhbhbnjohflphmfccbhmoi','mlfkmhibffpoleieiomjkekmjipdekhg',
 'gmopgnhbhiniibbiilmbjilcmgaocokj','ikdgincnppajmpmnhfheflannaiapmlm', 'dapjbgnjinbpoindlpdmhochffioedbn', 'didkfdopbffjkpolefhpcjkohcpalicd'
 'mmeijimgabbpbgpdklnllpncmdofkcpn','pbjikboenpfhbbejgkoklgkhjpfogcam', 'nkbihfbeogaeaoehlefnkodbefgpgknn','obhadkdgdffnnbdfpigjklinjhbkinfh']

pp = pd.read_excel('/xxxxxxxx/kmedoids2.xlsx','Sheet2')

dump_path='/xxxxxxxx'
dump_database_name='plugin.db'


db = sqlite3.connect(os.path.join(dump_path, dump_database_name))
extkeys = [*extensions.keys()]
attri = pd.read_sql(f'SELECT id, rate, rateCount, users, size FROM extensions where privacy != "not provided" and privacy != "None" and id IN (' + ",".join(["?"] * len(extkeys)) + ')',  params =extkeys, con=db)
pd.options.display.max_columns = None

shapiro = pd.DataFrame()

# fully loaded mode
for index, row in tqdm(full.iterrows(), total=full.shape[0]):
    extid = row.id
    base = extensions[extid]['target']+str(index%100//10)
    tmp ={
        "cluster": int(extensions[extid]['label']),# cluster
          "category":pp[pp.id==extid]["category"].values[0],# category
          "pp":int(extensions[extid]['pp']),
          "location":int(pp[pp.id==extid]["Location"].values[0]),# pp
          "user activity":int(pp[pp.id==extid]["User activity"].values[0]),
          "website content":int(pp[pp.id==extid]["Website content"].values[0]),
          "web history":int(pp[pp.id==extid]["Web history"].values[0]),
          "personally identifiable information":int(pp[pp.id==extid]["Personally identifiable information"].values[0]),
          "authentication information":int(pp[pp.id==extid]["Authentication information"].values[0]),
          "personal communications":int(pp[pp.id==extid]["Personal communications"].values[0]),
          "financial and payment information":int(pp[pp.id==extid]["Financial and payment information"].values[0]),
          "health information":int(pp[pp.id==extid]["Health information"].values[0]),
          "size":float(attri[attri.id==extid]["size"].apply(size_convert).values[0]), # size
          "users":float(attri[attri.id==extid]["users"].apply(user_convert).values[0]), # user #
          "rate": float(attri[attri.id==extid]["rate"]), # star
          "rateCount": int(attri[attri.id==extid]["rateCount"]), # rater #
          "login": 1 if extid in isLogin else 0, # is login required
          "grant": 1 if extid in isGrant else 0, # is grant required
          "inactive": 0, # test based on Generic website, login =1, grant=1
          "fullInactive": 0, # test based on Generic website, login =0, grant=0
          "testBase":base, #WebsiteType 
          "time":float(row.time),
          "loading": float(row.loading),
          "static": float(row.static)
         }
    shapiro = pd.concat([shapiro, pd.DataFrame([tmp])], ignore_index=True)
# notGrant
for index, row in tqdm(notGrant.iterrows(), total=notGrant.shape[0]):
    extid = row.id
    base = extensions[extid]['target']+str(index%100//10)
    tmp ={
        "cluster": int(extensions[extid]['label']),# cluster
          "category":pp[pp.id==extid]["category"].values[0],# category
          "pp":int(extensions[extid]['pp']),
          "location":int(pp[pp.id==extid]["Location"].values[0]),# pp
          "user activity":int(pp[pp.id==extid]["User activity"].values[0]),
          "website content":int(pp[pp.id==extid]["Website content"].values[0]),
          "web history":int(pp[pp.id==extid]["Web history"].values[0]),
          "personally identifiable information":int(pp[pp.id==extid]["Personally identifiable information"].values[0]),
          "authentication information":int(pp[pp.id==extid]["Authentication information"].values[0]),
          "personal communications":int(pp[pp.id==extid]["Personal communications"].values[0]),
          "financial and payment information":int(pp[pp.id==extid]["Financial and payment information"].values[0]),
          "health information":int(pp[pp.id==extid]["Health information"].values[0]),
          "size":float(attri[attri.id==extid]["size"].apply(size_convert).values[0]), # size
          "users":float(attri[attri.id==extid]["users"].apply(user_convert).values[0]), # user #
          "rate": float(attri[attri.id==extid]["rate"]), # star
          "rateCount": int(attri[attri.id==extid]["rateCount"]), # rater #
          "login": 1 if extid in isLogin else 0, # is login required
          "grant": 0, # is grant required
          "inactive": 0, # test based on Generic website, login =1, grant=1
          "fullInactive": 0, # test based on Generic website, login =0, grant=0
          "testBase":base, #WebsiteType 
          "time":float(row.time),
          "loading": float(row.loading),
          "static": float(row.static)
         }
    shapiro = pd.concat([shapiro, pd.DataFrame([tmp])], ignore_index=True)
# notLogin
for index, row in tqdm(notLogin.iterrows(), total=notLogin.shape[0]):
    extid = row.id
    base = extensions[extid]['target']+str(index%100//10)
    tmp ={
        "cluster": int(extensions[extid]['label']),# cluster
          "category":pp[pp.id==extid]["category"].values[0],# category
          "pp":int(extensions[extid]['pp']),
          "location":int(pp[pp.id==extid]["Location"].values[0]),# pp
          "user activity":int(pp[pp.id==extid]["User activity"].values[0]),
          "website content":int(pp[pp.id==extid]["Website content"].values[0]),
          "web history":int(pp[pp.id==extid]["Web history"].values[0]),
          "personally identifiable information":int(pp[pp.id==extid]["Personally identifiable information"].values[0]),
          "authentication information":int(pp[pp.id==extid]["Authentication information"].values[0]),
          "personal communications":int(pp[pp.id==extid]["Personal communications"].values[0]),
          "financial and payment information":int(pp[pp.id==extid]["Financial and payment information"].values[0]),
          "health information":int(pp[pp.id==extid]["Health information"].values[0]),
          "size":float(attri[attri.id==extid]["size"].apply(size_convert).values[0]), # size
          "users":float(attri[attri.id==extid]["users"].apply(user_convert).values[0]), # user #
          "rate": float(attri[attri.id==extid]["rate"]), # star
          "rateCount": int(attri[attri.id==extid]["rateCount"]), # rater #
          "login": 0, # is login required
          "grant": 1 if extid in isGrant else 0, # is grant required
          "inactive": 0, # test based on Generic website, login =1, grant=1
          "fullInactive": 0, # test based on Generic website, login =0, grant=0
          "testBase":base, #WebsiteType 
          "time":float(row.time),
          "loading": float(row.loading),
          "static": float(row.static)
         }
    shapiro = pd.concat([shapiro, pd.DataFrame([tmp])], ignore_index=True)
# inactive
for index, row in tqdm(inactive.iterrows(), total=inactive.shape[0]):
    extid = row.id
    base = 'Generic'+str(index%100//10)
    tmp ={
        "cluster": int(extensions[extid]['label']),# cluster
          "category":pp[pp.id==extid]["category"].values[0],# category
          "pp":int(extensions[extid]['pp']),
          "location":int(pp[pp.id==extid]["Location"].values[0]),# pp
          "user activity":int(pp[pp.id==extid]["User activity"].values[0]),
          "website content":int(pp[pp.id==extid]["Website content"].values[0]),
          "web history":int(pp[pp.id==extid]["Web history"].values[0]),
          "personally identifiable information":int(pp[pp.id==extid]["Personally identifiable information"].values[0]),
          "authentication information":int(pp[pp.id==extid]["Authentication information"].values[0]),
          "personal communications":int(pp[pp.id==extid]["Personal communications"].values[0]),
          "financial and payment information":int(pp[pp.id==extid]["Financial and payment information"].values[0]),
          "health information":int(pp[pp.id==extid]["Health information"].values[0]),
          "size":float(attri[attri.id==extid]["size"].apply(size_convert).values[0]), # size
          "users":float(attri[attri.id==extid]["users"].apply(user_convert).values[0]), # user #
          "rate": float(attri[attri.id==extid]["rate"]), # star
          "rateCount": int(attri[attri.id==extid]["rateCount"]), # rater #
          "login": 1 if extid in isLogin else 0, # is login required
          "grant": 1 if extid in isGrant else 0, # is grant required
          "inactive": 1, # test based on Generic website, login =1, grant=1
          "fullInactive": 0, # test based on Generic website, login =0, grant=0
          "testBase":base, #WebsiteType 
          "time":float(row.time),
          "loading": float(row.loading),
          "static": float(row.static)
         }
    shapiro = pd.concat([shapiro, pd.DataFrame([tmp])], ignore_index=True)
# fully Inactive
for index, row in tqdm(fullinactive.iterrows(), total=fullinactive.shape[0]):
    extid = row.id
    base = 'Generic'+str(index%100//10)
    tmp ={
        "cluster": int(extensions[extid]['label']),# cluster
          "category":pp[pp.id==extid]["category"].values[0],# category
          "pp":int(extensions[extid]['pp']),
          "location":int(pp[pp.id==extid]["Location"].values[0]),# pp
          "user activity":int(pp[pp.id==extid]["User activity"].values[0]),
          "website content":int(pp[pp.id==extid]["Website content"].values[0]),
          "web history":int(pp[pp.id==extid]["Web history"].values[0]),
          "personally identifiable information":int(pp[pp.id==extid]["Personally identifiable information"].values[0]),
          "authentication information":int(pp[pp.id==extid]["Authentication information"].values[0]),
          "personal communications":int(pp[pp.id==extid]["Personal communications"].values[0]),
          "financial and payment information":int(pp[pp.id==extid]["Financial and payment information"].values[0]),
          "health information":int(pp[pp.id==extid]["Health information"].values[0]),
          "size":float(attri[attri.id==extid]["size"].apply(size_convert).values[0]), # size
          "users":float(attri[attri.id==extid]["users"].apply(user_convert).values[0]), # user #
          "rate": float(attri[attri.id==extid]["rate"]), # star
          "rateCount": int(attri[attri.id==extid]["rateCount"]), # rater #
          "login": 0, # is login required
          "grant": 0, # is grant required
          "inactive": 0, # test based on Generic website, login =1, grant=1
          "fullInactive": 1, # test based on Generic website, login =0, grant=0
          "testBase":base, #WebsiteType 
          "time":float(row.time),
          "loading": float(row.loading),
          "static": float(row.static)
         }
    shapiro = pd.concat([shapiro, pd.DataFrame([tmp])], ignore_index=True)

In [ ]:
test =  shapiro.copy()

test.drop(['financial and payment information', 'health information'], axis=1, inplace=True)

one_hot2 = pd.get_dummies(test.testBase, prefix='base')
test.drop('category',axis = 1, inplace = True)
test.drop('testBase',axis = 1, inplace = True)
test.rename({'testBase': 'base'}, axis=1, inplace=True)
test = test.join(one_hot2)
test
from pyexcelerate import Workbook

save = [test.columns.tolist(), ] + test.values.tolist()
wb = Workbook()
wb.new_sheet('sheet1', data=save)
wb.save('/xxxxxxxx/shapiro.xlsx')